In [21]:
import pandas as pd

file_path = '../../real_data/trace/W2K8.TPCC.10-19-2007.12-51-PM.trace-filtered-target.txt'

In [22]:
#  read the file
trace = pd.read_csv(file_path, sep=' ')
trace.columns = ['ByteOffset', 'IOSize']
print(len(trace))
print(trace.head())

5899528
   ByteOffset  IOSize
0  3201773568    8192
1  3155169280    4096
2  3155156992    4096
3  3161305088    8192
4  3154132992    4096


In [ ]:
# declare dataframe variable to store converted trace
converted_trace = pd.DataFrame(columns=['ByteOffset', 'IOSize'])

# READ each line of the trace
for index, row in trace.iterrows():
    converted_list = []
    number_of_lba = row['IOSize'] / 8
    for i in range(int(number_of_lba)):
        converted_list.append({'ByteOffset': row['ByteOffset'] + i * 8, 'IOSize': 8})
    # converted_list to dataframe and concatenate to converted_trace
    converted_trace = pd.concat([converted_trace, pd.DataFrame(converted_list)], ignore_index=True)
converted_trace.columns = ['ByteOffset', 'IOSize']
print(len(converted_trace))
